In [4]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

#### Importing data 

In [5]:
# reaadinbg the data provided in csv format from data directory
Sales_features=pd.read_csv('data/Sales_features.csv')
Sales_stores=pd.read_csv('data/Sales_stores.csv')
Sales_test=pd.read_csv('data/Sales_test.csv')
Sales_train=pd.read_csv('data/Sales_train.csv')

In [6]:
# Merging Stores and Features into one 
dataset=Sales_features.merge(Sales_stores,how='inner',on='Store')

# Firstly we need to change the date format into year and week
dataset['year']=pd.DatetimeIndex(dataset['Date']).year ## Extracting 'year' data
dataset['week']=dataset['Date'].astype('datetime64[ns]').dt.isocalendar().week ## Extracting 'week' data
dataset.head()


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size,year,week
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,2010,5
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315,2010,6
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315,2010,7
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315,2010,8
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315,2010,9


In [7]:
# Droping the MarkDown1 to 5 features from  dataset as they contain too many outliers

dataset.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'],axis=1,inplace=True)

In [8]:
df_train=Sales_train.merge(dataset,how='inner',on=['Store','IsHoliday','Date']) # merging Sales_train and dataset.
df_test=Sales_test.merge(dataset,how='inner',on=['Store','IsHoliday','Date']) # merging Sales_test and dataset.


In [9]:
df_test.drop(['Date'],axis=1,inplace=True)
df_train.drop(['Date'],axis=1,inplace=True)
df_test.shape,df_train.shape

((115064, 11), (421570, 12))

In [22]:
# dividing traning dataset into X, y sets
y=df_train['Weekly_Sales']
X=df_train.drop('Weekly_Sales',axis=1)

In [11]:
X.describe(include='all').loc[['min', 'max']]
# temp, cpi,fuel, unemp, size

,Store,Dept,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,year,week
min,1.0,1.0,NaN,-2.06,2.472,126.064000,3.879,NaN,34875.0,2010.0,1.0
max,45.0,99.0,NaN,100.14,4.468,227.232807,14.313,NaN,219622.0,2012.0,52.0


In [26]:
type(pd.DataFrame(X.iloc[0]))

pandas.core.frame.DataFrame

In [13]:
# Changing the 'IsHoliday' feature from bool type to int
X.IsHoliday = X.IsHoliday.replace({True: 1, False: 0})

In [14]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [15]:
# fit and transform the preprocessorr above defined
X = preprocessor.fit_transform(X)

In [16]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((337256, 13), (84314, 13))

In [17]:
rfc=RandomForestRegressor()
rfc.fit(X_train,y_train)

RandomForestRegressor()

In [18]:
pre=rfc.predict(X_test)
test_model_score=r2_score(y_test,pre)
print(test_model_score)

0.9767473687758786


In [19]:
from sklearn.model_selection import GridSearchCV

def evaluate_model(X_train,y_train,X_test,y_test,models,param):    
    report={}
    for i in range(len(list(models))):
        model=list(models.values())[i]
        para=param[list(models.keys())[i]]
        
        gs=GridSearchCV(model,para,cv=3)
        gs.fit(X_train,y_train)

        model.set_params(**gs.best_params_)
        model.fit(X_train,y_train)

        #model.fit(X_train,y_train)

        y_train_pred=model.predict(X_train)
        y_test_pred=model.predict(X_test)

        train_model_score=r2_score(y_train,y_train_pred)
        test_model_score=r2_score(y_test,y_test_pred)

        report[list(models.keys())[i]]=test_model_score
    return report

In [20]:


            models={
                "Random Forest": RandomForestRegressor()
            }
            params={
                "Random Forest":{
                    # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                 
                    # 'max_features':['sqrt','log2',None],
                    'n_estimators': [8,16,32,64,128,256]
                }
                
            }

            model_report:dict=evaluate_model(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,models=models,param=params)


            # to get best model scpre from dictionary
            best_model_score=max(sorted(model_report.values()))

            # to get best model from dictionary
            best_model_name=list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]
            best_model=models[best_model_name]
            print(best_model)

KeyboardInterrupt: 

#### Create an Evaluating Function to give all metrics after model Training

In [ ]:
def evaluate_model(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    r2_square = r2_score(actual, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False)
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [ ]:
# Values of every model get the best 
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

### Random Forest Regressor

In [ ]:
rf=RandomForestRegressor()
rf = rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')